In [6]:
# %load simulate_motif_pairs.py
import os
import itertools as it
import pandas as pd
from collections import defaultdict, Counter
import json
import numpy as np
import time
import argparse
import tempfile

In [2]:
###############################################################################
# Setup the commandline interface
###############################################################################
inputpath = '../../../results/fimo/Aortic-VIC.GSE154513.Homo_Sapiens.H3K27ac.b1/summarize_results/summary.txt'
#savepath = args.savepath
sims = 10
account_dist = "No"
within_dist = 2000000
blacklist_path = '../../../results/blacklists/hg38-blacklist.v2.bed'

In [3]:
# load the input data

df = pd.read_csv(inputpath, sep = '\t', header=0)
df.shape

(6816, 10)

In [4]:
import pybedtools as pbt
pbt.set_bedtools_path('<path-to-bedtools-bin>')

In [7]:
# loading pbt's of loops and blacklisted regions
loops_pbt = pbt.BedTool(inputpath)
blacklist_pbt = pbt.BedTool(blacklist_path)

# extract loops with no blacklist regions
loops_without_blk = loops_pbt.pair_to_bed(blacklist_pbt, type='neither')
loops_without_blk = loops_without_blk.to_dataframe(header=0, disable_auto_names=True)
columns = ['chr1', 'start1', 'end1',
           'chr2', 'start2', 'end2',
           'motif_ID_1', 'motif_name_1', 'motif_ID_2',
           'motif_name_2']
loops_without_blk.columns = columns

# make a temp dir
temp_dir = tempfile.TemporaryDirectory()
    
# Create a temporary file within the temporary directory
loops_without_blk_fn = os.path.basename(inputpath) + '.no_blacklist_regions'
loops_without_blk_fn = os.path.join(temp_dir.name, loops_without_blk_fn)
loops_without_blk.to_csv(loops_without_blk_fn, sep='\t', header=None, index=None)

In [8]:
loops_without_blk.shape

(6773, 10)

In [57]:
loops_without_blk

,chr1,start1,end1,chr2,start2,end2,motif_ID_1,motif_name_1,motif_ID_2,motif_name_2
0,chr1,1690000,1695000,chr1,1745000,1750000,"MA1961.1,MA1513.1,MA1945.1,MA1934.1,MA1941.1,M...","PATZ1,KLF15,ETV5::FIGLA,ERF::FIGLA,ETV2::FIGLA...",NaN,NaN
1,chr1,2190000,2195000,chr1,2390000,2395000,"MA1972.1,MA1154.1,MA0039.4,MA1981.1,MA0599.1,M...","ZFP14,ZNF282,KLF4,ZNF530,KLF5,ETV5::FIGLA,EGR1...","MA1589.1,MA0645.1,MA1122.1,MA1961.1,MA1513.1,M...","ZNF140,ETV6,TFDP1,PATZ1,KLF15,ZNF320,CDX2,TEAD..."
2,chr1,2195000,2200000,chr1,2390000,2395000,"MA0114.4,MA0484.2,MA1976.1,MA0813.1,MA0815.1,M...","HNF4A,HNF4G,ZNF320,TFAP2B,TFAP2C,TFAP2A,REL,CT...","MA1589.1,MA0645.1,MA1122.1,MA1961.1,MA1513.1,M...","ZNF140,ETV6,TFDP1,PATZ1,KLF15,ZNF320,CDX2,TEAD..."
3,chr1,3050000,3055000,chr1,3365000,3370000,NaN,NaN,"MA1713.1,MA1977.1,MA1709.1","ZNF610,ZNF324,ZIM3"
4,chr1,8005000,8010000,chr1,8315000,8320000,"MA0149.1,MA1723.1,MA1596.1","EWSR1-FLI1,PRDM9,ZNF460","MA1653.1,MA1712.1,MA0609.2,MA0599.1,MA1513.1,M...","ZNF148,ZNF454,CREM,KLF5,KLF15,PATZ1,PRDM9,NEUROD1"
...,...,...,...,...,...,...,...,...,...,...
6768,chrX,154020000,154025000,chrX,154040000,154045000,MA1576.1,THRB,"MA1596.1,MA1149.1,MA1723.1,MA0597.2,MA1418.1,M...","ZNF460,RARA::RXRG,PRDM9,THAP1,IRF3,ZNF140,ZNF1..."
6769,chrX,154045000,154050000,chrX,154085000,154090000,"MA1596.1,MA1597.1","ZNF460,ZNF528",MA1596.1,ZNF460
6770,chrX,154375000,154380000,chrX,154395000,154400000,MA1102.2,CTCFL,"MA1711.1,MA1723.1","ZNF343,PRDM9"
6771,chrX,154395000,154400000,chrX,154425000,154430000,"MA1711.1,MA1723.1","ZNF343,PRDM9","MA1721.1,MA1522.1,MA1630.2,MA1653.1,MA1961.1,M...","ZNF93,MAZ,ZNF281,ZNF148,PATZ1,SP5,EBF3"


In [58]:

# drop any anchors with no motifs in either anchor (contains NaN)
filter_df =  loops_without_blk.loc[(~loops_without_blk['motif_name_1'].isna()) & (~loops_without_blk['motif_name_2'].isna())].reset_index(drop=True)

# add the anchor names
def create_anchor_name(sr, anchor_no):
    anchor_name = '{}-{}-{}'.format(sr[f'chr{anchor_no}'], sr[f'start{anchor_no}'], sr[f'end{anchor_no}'])
    return(anchor_name)
filter_df.loc[:, 'anchor_name1'] = filter_df.apply(create_anchor_name, args=[1], axis=1)
filter_df.loc[:, 'anchor_name2'] = filter_df.apply(create_anchor_name, args=[2], axis=1)


###############################################################################
# Counting Observed Motif Pairs
###############################################################################
print('# Counting Observed')

def sort_motifs(motifs):
    sorted_motifs = tuple(sorted(motifs))
    return sorted_motifs

# Set up counter
motif_pair_counter = Counter()

# Loop through all enteries in dataframe
uniq_motif_pairs = []
for num in range(len(filter_df)):

    # Get motifs 1
    motifs1 = filter_df.motif_name_1[num].split(',')
    
    # JR-Fix: I tried to remove duplicates but removed self motifs in the process.
    # need to fix this.
    # Get motifs 2; to avoid duplicates motif pairs, motifs in anchor1 are removed
    # from motifs in anchor 2
    motifs2 = filter_df.motif_name_2[num].split(',')
    
    # Form the combination of all motif1 and motif2 pairs by
    # using the cross product
    combos = it.product(motifs1, motifs2)
    
    # Organize motifs into alphabetical order and drop duplicates
    combos = [sort_motifs(tmotifs) for tmotifs in combos]
    combos = sorted(set(combos))
    
    # Record them in counter
    for p in combos:
        motif_pair_counter[p] += 1
        uniq_motif_pairs.append(p)
        
uniq_motif_pairs = list(set(uniq_motif_pairs))

###############################################################################
# Construct Anchor Slots and Dictionary
###############################################################################
# An anchor slot is composed of a genetic region corresponding to a loop anchor
# and its constitutent motifs as a list. These are stored in dictionarys as:
# {'chr1-10000-15000': ['ACT', 'GGG', 'CGT']}. These anchor slots are used for
# simulation steps where the anchor slots are randomized.
print('# Construct Anchor Slots')

# Initialize the anchor slots data structure
anchor_slots = {}
anchor_shuffle_dict = {}
anchor_ids = []
motifs_count_sims = []

# Loop through all entries in dataframe
for i, sr in filter_df.iterrows():

    # Create anchor ids
    anchor1_id = sr['anchor_name1']
    anchor2_id = sr['anchor_name2']

    # Create blank list of anchors for shuffling based on distance
    entry_shuffle = []
    
    # Get chromosome
    chr1 = sr['chr1']
    chr2 = sr['chr2']

    # Put anchor ids and slots into list
    if anchor1_id not in anchor_slots:
        
        # Create the current slot
        motifs1 = list(sr.motif_name_1.split(','))
        anchor_slots[anchor1_id] = list(set(motifs1))
        anchor_ids.append(anchor1_id)

        # For the given slot, determine what anchors can be shuffled with it. If distance has been
        # provided then this will be applied.
        # Create column in dataframe to get distance measure for anchor 1
        # JR note: how is this calculating distance? Why are we using start1 of filter_df with start1 of sr?
        # JR note: I think I see how the distance is being calculated but you would have to check the end as well!
        filter_df['tmp_dist'] = abs(filter_df['start1'] - sr['start1'])
        if account_dist == 'Yes':

            # Only keep anchors within certain distance and within certain chromosome
            tmp_dist_df = filter_df[(filter_df['tmp_dist'] <= within_dist) & \
                                    (filter_df['chr1'] == chr1) & \
                                    (filter_df['chr2'] == chr2)].copy()

        else:
            # Only keep motifs within specific chromosome
            tmp_dist_df = filter_df[(filter_df['chr1'] == chr1) & \
                                    (filter_df['chr2'] == chr2)].copy()
        
        # Create anchor id for anchors within distance of anchor 1
        tmp_dist_df['anchor1_id_name'] = tmp_dist_df['chr1'] + '-' + \
                                            tmp_dist_df['start1'].astype('str') + '-' + \
                                            tmp_dist_df['end1'].astype('str')
        entry_shuffle += list(tmp_dist_df['anchor1_id_name'])

        # Create list of anchors wihin distance
        anchor_shuffle_dict[anchor1_id] = entry_shuffle

        # Get summary stats of amount of motifs in anchors
        motifs_count_sims.append(len(entry_shuffle))
    
    if anchor2_id not in anchor_slots:
        
        motifs2 = list(sr.motif_name_2.split(','))
        anchor_slots[anchor2_id] = list(set(motifs2))
        anchor_ids.append(anchor2_id)
        
        # Create column in dataframe to get distance measure for anchor 1
        filter_df['tmp_dist'] = abs(filter_df['start2'] - sr['start2'])
        if account_dist == 'Yes':

            # Only keep anchors within certain distance and within certain chromosome
            tmp_dist_df = filter_df[(filter_df['tmp_dist'] <= within_dist) & \
                                    (filter_df['chr1'] == chr1) & \
                                    (filter_df['chr2'] == chr2)].copy()

        else:
            # Only keep motifs within specific chromosome
            tmp_dist_df = filter_df[(filter_df['chr1'] == chr1) & \
                                    (filter_df['chr2'] == chr2)].copy()

        # Create anchor id for anchors within distance of anchor 2
        tmp_dist_df['anchor2_id_name'] = tmp_dist_df['chr2'] + '-' + \
                                            tmp_dist_df['start2'].astype('str') + '-' + \
                                            tmp_dist_df['end2'].astype('str')
        entry_shuffle += list(tmp_dist_df['anchor2_id_name'])

        # Create list of anchors within distance
        anchor_shuffle_dict[anchor2_id] = entry_shuffle

        # Get summary stats of amount of motifs in anchors
        motifs_count_sims.append(len(entry_shuffle))

# Counting Observed
# Construct Anchor Slots


In [59]:
###############################################################################
# Conduct Simulations
###############################################################################
print('# Conduct Simulations')

sim_results = []
sim_idx = 0
num_loops = len(filter_df)

while sim_idx < sims:

    sims_motif_pair_counter = Counter()
    for anchor_name1, anchor_name2 in filter_df[['anchor_name1', 'anchor_name2']].values:

        # Simulate anchors based on chromsome and distance
        sim_anchors1 = np.random.choice(anchor_shuffle_dict[anchor_name], size=1, replace=True) 
        sim_anchors2 = np.random.choice(anchor_shuffle_dict[anchor_name], size=1, replace=True) 
        
        # Get motifs from simulated anchors
        # To remove duplicates motif pairs, motifs in anchor1 are removed from motifs in anchor 2
        sim_motifs1 = anchor_slots[sim_anchors1[0]]
        sim_motifs2 = anchor_slots[sim_anchors2[0]]
    
        # Form the combination of all motif1 and motif2 pairs by
        # using the cross product
        shuffle_combos = it.product(sim_motifs1, sim_motifs2)

        # Organize motifs into alphabetical order
        shuffle_combos = [sort_motifs(tmotifs) for tmotifs in shuffle_combos]
        shuffle_combos = sorted(set(shuffle_combos))

        # Record them in counter
        for p in shuffle_combos:
            sims_motif_pair_counter[p] += 1

    sim_results.append(sims_motif_pair_counter)
    sim_idx += 1
    

# Conduct Simulations


NameError: name 'anchor_name' is not defined

In [47]:
sims = 1000

In [48]:
###############################################################################
# Conduct Simulations
###############################################################################
print('# Conduct Simulations')

sim_results = [Counter() for sim in range(sims)]
sim_idx = 0
num_loops = len(filter_df)

sims_motif_pair_counter = Counter()
for anchor_name1, anchor_name2 in filter_df[['anchor_name1', 'anchor_name2']].values:

    # Simulate anchors based on chromsome and distance
    sim_anchors1 = np.random.choice(anchor_shuffle_dict[anchor_name], size=sims, replace=True) 
    sim_anchors2 = np.random.choice(anchor_shuffle_dict[anchor_name], size=sims, replace=True) 
    
    sim_idx = 0
    for (slot1, slot2) in zip(sim_anchors1, sim_anchors2):
    
        # Get motifs from simulated anchors
        # To remove duplicates motif pairs, motifs in anchor1 are removed from motifs in anchor 2
        sim_motifs1 = anchor_slots[slot1]
        sim_motifs2 = anchor_slots[slot2]

        # Form the combination of all motif1 and motif2 pairs by
        # using the cross product
        shuffle_combos = it.product(sim_motifs1, sim_motifs2)

        # Organize motifs into alphabetical order
        shuffle_combos = [sort_motifs(tmotifs) for tmotifs in shuffle_combos]
        shuffle_combos = sorted(set(shuffle_combos))

        # Record them in counter
        for p in shuffle_combos:
            sim_results[sim_idx][p] += 1 
        
        # Update counter
        sim_idx += 1
                    

# Conduct Simulations


In [49]:
len(sim_results)

1000

In [50]:
sim_results[0]

Counter({('ZNF460', 'ZNF93'): 253,
         ('PRDM9', 'ZNF460'): 242,
         ('PRDM9', 'ZNF93'): 232,
         ('PRDM9', 'RARA::RXRG'): 186,
         ('PATZ1', 'PRDM9'): 180,
         ('PATZ1', 'ZNF460'): 167,
         ('PATZ1', 'ZNF93'): 164,
         ('RARA::RXRG', 'ZNF93'): 164,
         ('PRDM9', 'SP5'): 164,
         ('RARA::RXRG', 'ZNF460'): 153,
         ('PRDM9', 'PRDM9'): 147,
         ('SP5', 'ZNF460'): 146,
         ('PRDM9', 'ZNF528'): 139,
         ('ZNF528', 'ZNF93'): 138,
         ('ZNF460', 'ZNF701'): 133,
         ('PRDM9', 'RARA'): 133,
         ('SP5', 'ZNF93'): 132,
         ('PRDM9', 'ZNF454'): 125,
         ('ZNF93', 'ZNF93'): 123,
         ('PRDM9', 'RARA::RXRA'): 120,
         ('MAZ', 'PRDM9'): 120,
         ('PRDM9', 'ZNF148'): 120,
         ('RARA', 'ZNF460'): 119,
         ('EWSR1-FLI1', 'ZNF460'): 115,
         ('MAZ', 'ZNF460'): 114,
         ('ZNF148', 'ZNF460'): 114,
         ('PATZ1', 'RARA::RXRG'): 112,
         ('RARA::RXRA', 'ZNF460'): 111,
        

In [11]:
###############################################################################
# Save json file to output
###############################################################################
print('# Save json file to output')

# Keys are from observed
# Aggregate simulations into one list only for observed motif pairs   
mp_sim_aggs = defaultdict(list)
for sim_idx in range(sims):
    for mp in uniq_motif_pairs:
        mp_sim_aggs[str(mp)].append(sim_results[sim_idx][mp])

# # Save json
# json_d = json.dumps(mp_sim_aggs)
# with open(savepath, 'w') as fw:
#     fw.write(json_d)

# Counting Observed
# Construct Anchor Slots
# Conduct Simulations


KeyError: ('anchor_name1', 'anchor_name2')